# **Setup**

- https://towardsdatascience.com/practical-introduction-to-polars-8d9cdca350f1 

In [123]:
from dataclasses import dataclass
from datetime import datetime, timedelta
from random import choice, gauss, randrange, seed
from typing import Any, Dict

import pandas as pd
import polars as pl

seed(42)

# **Index**

1. Data preparation and DataFrame creation.
2. Summary statistics of the DataFrame.
3. Retrieving the first five records.
4. Renaming columns.
5. Changing column types.
6. Filling missing values.
7. Removing missing values.
8. Removing duplicate records.
9. Filtering data.
10. Selecting the required columns.
11. Grouping data.
12. Merging data with another DataFrame.
13. Calculating a new column.
14. Creating a Pivot table.

# **01 Generate Demo Data**

In [124]:
base_time= datetime(2024, 8, 31, 0, 0, 0, 0)
base_time

datetime.datetime(2024, 8, 31, 0, 0)

In [125]:
user_actions_data = [
    {
        "OnlineStore": choice(["Shop1", "Shop2", "Shop3"]),
        "product": choice(["0001", "0002", "0003"]),
        "quantity": choice([1.0, 2.0, 3.0]),
        "Action type": ("purchase" if gauss() > 0.6 else "view"),
        "Action_time": base_time - timedelta(minutes=randrange(1_000_000)),
    }
    for x in range(1_000_000)
]

# Data Preview 
user_actions_data[:5]

[{'OnlineStore': 'Shop3',
  'product': '0001',
  'quantity': 1.0,
  'Action type': 'view',
  'Action_time': datetime.datetime(2024, 5, 21, 9, 24)},
 {'OnlineStore': 'Shop3',
  'product': '0001',
  'quantity': 3.0,
  'Action type': 'view',
  'Action_time': datetime.datetime(2023, 3, 10, 15, 54)},
 {'OnlineStore': 'Shop3',
  'product': '0001',
  'quantity': 3.0,
  'Action type': 'view',
  'Action_time': datetime.datetime(2024, 3, 24, 19, 2)},
 {'OnlineStore': 'Shop1',
  'product': '0003',
  'quantity': 3.0,
  'Action type': 'view',
  'Action_time': datetime.datetime(2024, 8, 11, 16, 16)},
 {'OnlineStore': 'Shop3',
  'product': '0001',
  'quantity': 3.0,
  'Action type': 'view',
  'Action_time': datetime.datetime(2024, 3, 23, 11, 32)}]

In [126]:
# Generate Demo Data
corrupted_data = [
    {
        "OnlineStore": choice(["Shop1", "Shop2", "Shop3"]),
        "product": choice(["0001", None]),
        "quantity": choice([1.0, None]),
        "Action type": ("purchase" if gauss() > 0.6 else "view"),
        "Action_time": base_time - timedelta(minutes=randrange(1_000)),
    }
    for x in range(1_000)
]

# Preview Corrupted Data
corrupted_data[:5]

[{'OnlineStore': 'Shop1',
  'product': '0001',
  'quantity': 1.0,
  'Action type': 'view',
  'Action_time': datetime.datetime(2024, 8, 30, 14, 11)},
 {'OnlineStore': 'Shop3',
  'product': '0001',
  'quantity': None,
  'Action type': 'view',
  'Action_time': datetime.datetime(2024, 8, 30, 17, 9)},
 {'OnlineStore': 'Shop3',
  'product': '0001',
  'quantity': 1.0,
  'Action type': 'purchase',
  'Action_time': datetime.datetime(2024, 8, 30, 11, 23)},
 {'OnlineStore': 'Shop2',
  'product': None,
  'quantity': None,
  'Action type': 'view',
  'Action_time': datetime.datetime(2024, 8, 30, 19, 53)},
 {'OnlineStore': 'Shop3',
  'product': None,
  'quantity': 1.0,
  'Action type': 'purchase',
  'Action_time': datetime.datetime(2024, 8, 30, 9, 36)}]

In [127]:
product_catalog_data = {"product_id": ["0001", "0002", "0003"], "price": [100, 25, 80]}

# Preview Product Catalog Data
product_catalog_data

{'product_id': ['0001', '0002', '0003'], 'price': [100, 25, 80]}

## **Create Dataframes**

In [128]:
# Pandas
user_actions_pd_df = pd.DataFrame(user_actions_data)
corrupted_pd_df = pd.DataFrame(corrupted_data)
product_catalog_pd_df = pd.DataFrame(product_catalog_data)

# Polars
user_actions_pl_df = pl.DataFrame(user_actions_data)
corrupted_pl_df = pl.DataFrame(corrupted_data)
product_catalog_pl_df = pl.DataFrame(product_catalog_data)

## **Concatenate Dataframes**

In [129]:
# Pandas
user_actions_pd_df = pd.concat([user_actions_pd_df, corrupted_pd_df])

# Polars
user_actions_pl_df = pl.concat([user_actions_pl_df, corrupted_pl_df])

# **02 Summary Statistics**

In [130]:
# Pandas
user_actions_pd_df.describe(include='all')

,OnlineStore,product,quantity,Action type,Action_time
count,1001000,1000492,1.000510e+06,1001000,1001000
unique,3,3,NaN,2,NaN
top,Shop3,0001,NaN,view,NaN
freq,333931,333963,NaN,726623,NaN
mean,NaN,NaN,1.998925e+00,NaN,2023-09-19 03:24:30.981699072
min,NaN,NaN,1.000000e+00,NaN,2022-10-06 13:23:00
25%,NaN,NaN,1.000000e+00,NaN,2023-03-29 03:08:45
50%,NaN,NaN,2.000000e+00,NaN,2023-09-19 06:48:30
75%,NaN,NaN,3.000000e+00,NaN,2024-03-11 03:01:15
max,NaN,NaN,3.000000e+00,NaN,2024-08-30 23:58:00


In [131]:
# Polars
user_actions_pl_df.describe()

statistic,OnlineStore,product,quantity,Action type,Action_time
str,str,str,f64,str,str
"""count""","""1001000""","""1000492""",1.00051e6,"""1001000""","""1001000"""
"""null_count""","""0""","""508""",490.0,"""0""","""0"""
"""mean""",null,null,1.998925,null,"""2023-09-19 03:24:30.981698"""
"""std""",null,null,0.816446,null,null
"""min""","""Shop1""","""0001""",1.0,"""purchase""","""2022-10-06 13:23:00"""
"""25%""",null,null,1.0,null,"""2023-03-29 03:09:00"""
"""50%""",null,null,2.0,null,"""2023-09-19 06:49:00"""
"""75%""",null,null,3.0,null,"""2024-03-11 03:01:00"""
"""max""","""Shop3""","""0003""",3.0,"""view""","""2024-08-30 23:58:00"""


## **Dataframe Info**

In [132]:
# Pandas
user_actions_pd_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1001000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   OnlineStore  1001000 non-null  object        
 1   product      1000492 non-null  object        
 2   quantity     1000510 non-null  float64       
 3   Action type  1001000 non-null  object        
 4   Action_time  1001000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 45.8+ MB


In [133]:
# Polars - Glimpse Function
user_actions_pl_df.glimpse()

Rows: 1001000
Columns: 5
$ OnlineStore          <str> 'Shop3', 'Shop3', 'Shop3', 'Shop1', 'Shop3', 'Shop2', 'Shop1', 'Shop2', 'Shop1', 'Shop2'
$ product              <str> '0001', '0001', '0001', '0003', '0001', '0003', '0001', '0001', '0002', '0003'
$ quantity             <f64> 1.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 1.0, 2.0, 1.0
$ Action type          <str> 'view', 'view', 'view', 'view', 'view', 'view', 'view', 'view', 'view', 'view'
$ Action_time <datetime[μs]> 2024-05-21 09:24:00, 2023-03-10 15:54:00, 2024-03-24 19:02:00, 2024-08-11 16:16:00, 2024-03-23 11:32:00, 2023-01-19 14:11:00, 2024-03-27 05:08:00, 2023-11-28 08:18:00, 2023-03-18 15:01:00, 2022-10-29 09:44:00



# **03 View First 5 Records**

In [134]:
# Pandas
user_actions_pd_df.head()

,OnlineStore,product,quantity,Action type,Action_time
0,Shop3,0001,1.0,view,2024-05-21 09:24:00
1,Shop3,0001,3.0,view,2023-03-10 15:54:00
2,Shop3,0001,3.0,view,2024-03-24 19:02:00
3,Shop1,0003,3.0,view,2024-08-11 16:16:00
4,Shop3,0001,3.0,view,2024-03-23 11:32:00


In [135]:
# Polars
user_actions_pl_df.head()

OnlineStore,product,quantity,Action type,Action_time
str,str,f64,str,datetime[μs]
"""Shop3""","""0001""",1.0,"""view""",2024-05-21 09:24:00
"""Shop3""","""0001""",3.0,"""view""",2023-03-10 15:54:00
"""Shop3""","""0001""",3.0,"""view""",2024-03-24 19:02:00
"""Shop1""","""0003""",3.0,"""view""",2024-08-11 16:16:00
"""Shop3""","""0001""",3.0,"""view""",2024-03-23 11:32:00


# **04 Rename Columns**

In [136]:
# Pandas
user_actions_pd_df = user_actions_pd_df.rename(
    columns={
        "OnlineStore": "online_store",
        "product": "product_id",
        "Action type": "action_type",
        "Action_time": "action_dt",
    }
)

user_actions_pd_df.columns

Index(['online_store', 'product_id', 'quantity', 'action_type', 'action_dt'], dtype='object')

In [137]:
# Polars
user_actions_pl_df = user_actions_pl_df.rename(
    {
        "OnlineStore": "online_store",
        "product": "product_id",
        "Action type": "action_type",
        "Action_time": "action_dt",
    }
)

user_actions_pl_df.columns

['online_store', 'product_id', 'quantity', 'action_type', 'action_dt']

# **05 Change Column Data Types**

In [138]:
# Pandas
user_actions_pd_df = user_actions_pd_df.astype({"quantity": "Int64"})

user_actions_pd_df.dtypes

online_store            object
product_id              object
quantity                 Int64
action_type             object
action_dt       datetime64[ns]
dtype: object

In [139]:
# Polars
user_actions_pl_df = user_actions_pl_df.cast({"quantity": pl.Int32})

# Get Data Types
user_actions_pl_df.dtypes

[String, String, Int32, String, Datetime(time_unit='us', time_zone=None)]

# **XX Data Size**

In [140]:
# Pandas
user_actions_pd_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1001000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   online_store  1001000 non-null  object        
 1   product_id    1000492 non-null  object        
 2   quantity      1000510 non-null  Int64         
 3   action_type   1001000 non-null  object        
 4   action_dt     1001000 non-null  datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), object(3)
memory usage: 46.8+ MB


In [141]:
# Polars
user_actions_pl_df.estimated_size("mb")

24.91054630279541

# **06 Fill Missing Values**

In [142]:
user_actions_pd_df.info()

# Pandas
user_actions_pd_df["quantity"].fillna(0, inplace=True)
display(user_actions_pd_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1001000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   online_store  1001000 non-null  object        
 1   product_id    1000492 non-null  object        
 2   quantity      1000510 non-null  Int64         
 3   action_type   1001000 non-null  object        
 4   action_dt     1001000 non-null  datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), object(3)
memory usage: 46.8+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 1001000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   online_store  1001000 non-null  object        
 1   product_id    1000492 non-null  object        
 2   quantity      1001000 non-null  Int64         
 3   action_type   1001000 non-null  object        
 4   action_dt     1001000 non-null  datetime64[

C:\Users\aksha\AppData\Local\Temp\ipykernel_12664\370503049.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  user_actions_pd_df["quantity"].fillna(0, inplace=True)


None

In [143]:
# Polars
user_actions_pl_df = user_actions_pl_df.with_columns(pl.col("quantity").fill_null(0))

# Describe
user_actions_pl_df.describe()

statistic,online_store,product_id,quantity,action_type,action_dt
str,str,str,f64,str,str
"""count""","""1001000""","""1000492""",1.001e6,"""1001000""","""1001000"""
"""null_count""","""0""","""508""",0.0,"""0""","""0"""
"""mean""",null,null,1.997946,null,"""2023-09-19 03:24:30.981698"""
"""std""",null,null,0.817442,null,null
"""min""","""Shop1""","""0001""",0.0,"""purchase""","""2022-10-06 13:23:00"""
"""25%""",null,null,1.0,null,"""2023-03-29 03:09:00"""
"""50%""",null,null,2.0,null,"""2023-09-19 06:49:00"""
"""75%""",null,null,3.0,null,"""2024-03-11 03:01:00"""
"""max""","""Shop3""","""0003""",3.0,"""view""","""2024-08-30 23:58:00"""


# **07 Remove Missing Values**

In [144]:
# Pandas
user_actions_pd_df.dropna(subset=["product_id"], inplace=True)

user_actions_pd_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000492 entries, 0 to 999
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   online_store  1000492 non-null  object        
 1   product_id    1000492 non-null  object        
 2   quantity      1000492 non-null  Int64         
 3   action_type   1000492 non-null  object        
 4   action_dt     1000492 non-null  datetime64[ns]
dtypes: Int64(1), datetime64[ns](1), object(3)
memory usage: 46.8+ MB


In [145]:
# Polars
user_actions_pl_df = user_actions_pl_df.drop_nulls(subset=["product_id"])

# **08 Remove Duplicates**

In [146]:
# Pandas
user_actions_pd_df.drop_duplicates(
    subset=["online_store", "action_type", "action_dt"],
    keep="last",
    inplace=True,
)

In [147]:
# Polars
user_actions_pl_df = user_actions_pl_df.unique(
    subset=["online_store", "action_type", "action_dt"],
    keep="last",
)

# **09 Filtering Data**

In [148]:
# Pandas
user_actions_pd_df = user_actions_pd_df.loc[
    user_actions_pd_df["action_type"] == "purchase"
]

user_actions_pd_df

,online_store,product_id,quantity,action_type,action_dt
11,Shop2,0003,1,purchase,2023-04-05 22:43:00
15,Shop2,0003,3,purchase,2023-05-17 02:46:00
16,Shop1,0003,3,purchase,2024-05-04 00:21:00
17,Shop2,0002,2,purchase,2022-10-27 03:38:00
19,Shop1,0001,1,purchase,2023-01-22 18:49:00
...,...,...,...,...,...
978,Shop2,0001,1,purchase,2024-08-30 13:49:00
982,Shop3,0001,0,purchase,2024-08-30 14:35:00
985,Shop3,0001,1,purchase,2024-08-30 18:40:00
988,Shop2,0001,1,purchase,2024-08-30 07:28:00


In [149]:
# Polars
user_actions_pl_df = user_actions_pl_df.filter(
    pl.col("action_type") == "purchase"
)

user_actions_pl_df.head(5)

online_store,product_id,quantity,action_type,action_dt
str,str,i32,str,datetime[μs]
"""Shop1""","""0002""",3,"""purchase""",2023-11-02 07:39:00
"""Shop2""","""0003""",1,"""purchase""",2022-10-19 16:03:00
"""Shop1""","""0003""",2,"""purchase""",2023-06-18 18:41:00
"""Shop2""","""0001""",2,"""purchase""",2023-11-17 14:59:00
"""Shop1""","""0002""",2,"""purchase""",2023-01-26 20:03:00


# **10 Select Specific Columns**

In [150]:
# Pandas
user_actions_pd_df_01 = user_actions_pd_df[
    ["online_store", "action_type", "product_id", "quantity"]
]

user_actions_pd_df_01.head()

,online_store,action_type,product_id,quantity
11,Shop2,purchase,0003,1
15,Shop2,purchase,0003,3
16,Shop1,purchase,0003,3
17,Shop2,purchase,0002,2
19,Shop1,purchase,0001,1


In [151]:
# Polars
user_actions_pl_df_01 = user_actions_pl_df.select(
    "online_store", "action_type", "product_id", "quantity"
)

user_actions_pl_df_01.head()   

online_store,action_type,product_id,quantity
str,str,str,i32
"""Shop1""","""purchase""","""0002""",3
"""Shop2""","""purchase""","""0003""",1
"""Shop1""","""purchase""","""0003""",2
"""Shop2""","""purchase""","""0001""",2
"""Shop1""","""purchase""","""0002""",2


# **11 Grouping Data**

In [152]:
# Pandas
user_actions_pd_df = (
    user_actions_pd_df.groupby(["online_store", "product_id", "action_type"])
    .agg({"quantity": "sum"})
    .reset_index()
)

user_actions_pd_df.head()

,online_store,product_id,action_type,quantity
0,Shop1,0001,purchase,57772
1,Shop1,0002,purchase,58015
2,Shop1,0003,purchase,58242
3,Shop2,0001,purchase,58256
4,Shop2,0002,purchase,58648


In [153]:
# Polars
user_actions_pl_df = (
    user_actions_pl_df.group_by(["online_store", "product_id", "action_type"])
    .agg(pl.col("quantity").sum())
)

user_actions_pl_df.head()

online_store,product_id,action_type,quantity
str,str,str,i32
"""Shop2""","""0001""","""purchase""",58256
"""Shop1""","""0002""","""purchase""",58015
"""Shop2""","""0003""","""purchase""",58458
"""Shop1""","""0001""","""purchase""",57772
"""Shop2""","""0002""","""purchase""",58648


# **12 Merge Dataframes**

In [154]:
# Pandas - Merge & Join methods
user_actions_pd_df = user_actions_pd_df.merge(product_catalog_pd_df, on='product_id')

user_actions_pd_df.head()

,online_store,product_id,action_type,quantity,price
0,Shop1,0001,purchase,57772,100
1,Shop1,0002,purchase,58015,25
2,Shop1,0003,purchase,58242,80
3,Shop2,0001,purchase,58256,100
4,Shop2,0002,purchase,58648,25


In [155]:
# Polars - Join method
user_actions_pl_df = user_actions_pl_df.join(product_catalog_pl_df, on='product_id')

user_actions_pl_df.tail()

online_store,product_id,action_type,quantity,price
str,str,str,i32,i64
"""Shop2""","""0002""","""purchase""",58648,25
"""Shop3""","""0002""","""purchase""",58326,25
"""Shop3""","""0003""","""purchase""",59107,80
"""Shop1""","""0003""","""purchase""",58242,80
"""Shop3""","""0001""","""purchase""",57891,100


# **13 Add New Column**

In [156]:
# Pandas
user_actions_pd_df["total"] = (
    user_actions_pd_df["price"] * user_actions_pd_df["quantity"]
)
user_actions_pd_df = user_actions_pd_df[["online_store", "action_type", "total"]]
user_actions_pd_df.head()

,online_store,action_type,total
0,Shop1,purchase,5777200
1,Shop1,purchase,1450375
2,Shop1,purchase,4659360
3,Shop2,purchase,5825600
4,Shop2,purchase,1466200


In [157]:
# Polars
user_actions_pl_df = user_actions_pl_df.with_columns((pl.col("price") * pl.col("quantity")).alias("total"))

user_actions_pl_df_01 = user_actions_pl_df.select("online_store", "action_type", "total")
user_actions_pl_df_01.head()

online_store,action_type,total
str,str,i64
"""Shop2""","""purchase""",5825600
"""Shop1""","""purchase""",1450375
"""Shop2""","""purchase""",4676640
"""Shop1""","""purchase""",5777200
"""Shop2""","""purchase""",1466200


In [158]:
# Polars Option 02
user_actions_pl_df_02 = user_actions_pl_df.select("online_store","action_type",
    (pl.col("price") * pl.col("quantity")).alias("total 02"),)

user_actions_pl_df_02.head()

online_store,action_type,total 02
str,str,i64
"""Shop2""","""purchase""",5825600
"""Shop1""","""purchase""",1450375
"""Shop2""","""purchase""",4676640
"""Shop1""","""purchase""",5777200
"""Shop2""","""purchase""",1466200


# **14 Pivot Table**

In [159]:
# Pandas
result_pd = user_actions_pd_df.pivot_table(columns="online_store",index="action_type",values="total",aggfunc="sum",)

result_pd

online_store,Shop1,Shop2,Shop3
action_type,,,
purchase,11886935,11968440,11975810


In [160]:
# Polars
result_pl = user_actions_pl_df.pivot(on="online_store", index="action_type", 
                                    values="total", aggregate_function="sum",)

result_pl

action_type,Shop2,Shop1,Shop3
str,i64,i64,i64
"""purchase""",11968440,11886935,11975810
